<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [1]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [2]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [3]:
display(apps)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [4]:
apps = apps.drop(columns=['Type', 'Price', 'Category','Last Updated', 'Current Ver'])
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19M,"10,000+",Everyone,Art & Design,4.0.3 and up
1,Coloring book moana,3.9,967,14M,"500,000+",Everyone,Art & Design;Pretend Play,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8.7M,"5,000,000+",Everyone,Art & Design,4.0.3 and up
3,Sketch - Draw & Paint,4.5,215644,25M,"50,000,000+",Teen,Art & Design,4.2 and up
4,Pixel Draw - Number Art Coloring Book,4.3,967,2.8M,"100,000+",Everyone,Art & Design;Creativity,4.4 and up


In [5]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','a', regex=True).str.replace('a','.', 1, regex=True).str.replace('a','', regex=True)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})

C:\Users\rafal\AppData\Local\Temp\ipykernel_20416\2251694235.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\rafal\AppData\Local\Temp\ipykernel_20416\2251694235.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\rafal\AppData\Local\Temp\ipykernel_20416\2251694235.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


In [6]:
class DataClean:
    @staticmethod
    def splitGenres(x):
        genresConnected = x["Genres"].unique().tolist()
        genres = set(())
        for i in genresConnected:
            afterSplit = i.split(";")
            for j in afterSplit:
                if j!='February 11, 2018':
                    genres.add(j)
        for i in list(genres):
            x[i] = [0 for x in range(x.shape[0])]
        for index in x.index:
            values = x.loc[index,'Genres'].split(";")
            for i in values:
                x.loc[index,i] = 1
        return x

In [7]:
apps = DataClean.splitGenres(apps)
apps.head()
apps.drop(columns=['Genres'])

,App,Rating,Reviews,Size,Installs,Content Rating,Android Ver,Action,Tools,Dating,...,Music,Finance,Board,Racing,Lifestyle,Auto & Vehicles,Music & Video,Parenting,Beauty,Brain Games
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159.0,19.0,10000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,3.9,967.0,14.0,500000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510.0,8.7,5000000.0,Everyone,4.03,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sketch - Draw & Paint,4.5,215644.0,25.0,50000000.0,Teen,4.20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,4.3,967.0,2.8,100000.0,Everyone,4.40,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10832,FR Tides,3.8,1195.0,582.0,100000.0,Everyone,2.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10833,Chemin (fr),4.8,44.0,619.0,1000.0,Everyone,2.20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10834,FR Calculator,4.0,7.0,2.6,500.0,Everyone,4.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10836,Sya9a Maroc - FR,4.5,38.0,53.0,5000.0,Everyone,4.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [9]:
DataProcessing.shuffle(apps)
apps = apps.reset_index(drop=True)

In [10]:
DataProcessing.normalization(apps)

In [11]:
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Action,Tools,...,Music,Finance,Board,Racing,Lifestyle,Auto & Vehicles,Music & Video,Parenting,Beauty,Brain Games
0,CJ Fallon eBook Reader,0.225,0.000002,0.013092,0.000005,Everyone,Education,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Trading 212 - Forex, Stocks, CFDs",0.825,0.000837,0.036395,0.005000,Everyone,Finance,0.485714,0,0,...,0,1,0,0,0,0,0,0,0,0
2,MLB Ballpark,0.725,0.000123,0.031219,0.001000,Everyone,Sports,0.457143,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Facetune - For Free,0.850,0.001104,0.047331,0.001000,Everyone,Photography,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AE 3D Moto 3,0.725,0.000058,0.025176,0.000100,Everyone 10+,Racing,0.185714,0,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.999)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Action,Tools,...,Music,Finance,Board,Racing,Lifestyle,Auto & Vehicles,Music & Video,Parenting,Beauty,Brain Games
0,Shooting King,0.850,5.740715e-03,0.068479,9.999999e-03,Everyone 10+,Sports,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Talking Angela,0.675,4.072454e-02,0.051360,1.000000e-01,Everyone,Entertainment,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"BestCam Selfie-selfie, beauty camera, photo ed...",0.725,3.871351e-05,0.020141,4.999990e-04,Everyone,Beauty,0.432857,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Learn Artificial Intelligence,0.900,5.791434e-07,0.003223,9.999000e-06,Everyone,Education,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Co Checker,0.225,2.450222e-07,0.001007,9.990000e-07,Everyone,Tools,0.471429,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Series Valley of the Wolves full of part,0.900,1.626057e-05,0.010070,4.999900e-05,Everyone,Events,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Hulu: Stream TV, Movies & more",0.750,7.121014e-03,0.036395,9.999999e-03,Teen,Entertainment,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
7,How to Write CV,0.900,5.412764e-06,0.003122,9.999900e-05,Everyone,Books & Reference,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
apps["Content Rating"].unique().tolist()

['Everyone',
 'Everyone 10+',
 'Teen',
 'Mature 17+',
 'Adults only 18+',
 'Unrated']

In [14]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [15]:
validatingSet.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Action,Tools,...,Music,Finance,Board,Racing,Lifestyle,Auto & Vehicles,Music & Video,Parenting,Beauty,Brain Games
0,Shooting King,0.850,5.740715e-03,0.068479,9.999999e-03,Teen,Sports,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Talking Angela,0.675,4.072454e-02,0.051360,1.000000e-01,Teen,Entertainment,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"BestCam Selfie-selfie, beauty camera, photo ed...",0.725,3.871351e-05,0.020141,4.999990e-04,Teen,Beauty,0.432857,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Learn Artificial Intelligence,0.900,5.791434e-07,0.003223,9.999000e-06,Teen,Education,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Co Checker,0.225,2.450222e-07,0.001007,9.990000e-07,Teen,Tools,0.471429,0,1,...,0,0,0,0,0,0,0,0,0,0


<h2>k-nearest neighbors algorithm</h2>

In [16]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return sum**(1/2)
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNN.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h2>Apps Recommendation</h2>

In [17]:
# categoryList = validatingSet['Category'].unique().tolist()
# avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
# avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
# avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
# avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
# maxAndroidVer = max(validatingSet['Android Ver'])

# avgApp = pd.DataFrame(columns=['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Content Rating', 'Android Ver'])    
# avgApp.loc[0] = ['a', categoryList, avgRating, avgReviews, avgSize, avgInstalls, 'c', maxAndroidVer]
recommendedApps = KNN.clustering(trainingSet.copy(), validatingSet.iloc[0])
recommendedApps

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Action,Tools,...,Finance,Board,Racing,Lifestyle,Auto & Vehicles,Music & Video,Parenting,Beauty,Brain Games,distance
0,CJ Fallon eBook Reader,0.225,0.000002,0.013092,0.000005,Everyone,Education,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,1.547199
1,"Trading 212 - Forex, Stocks, CFDs",0.825,0.000837,0.036395,0.005000,Everyone,Finance,0.485714,0,0,...,1,0,0,0,0,0,0,0,0,10.000000
2,MLB Ballpark,0.725,0.000123,0.031219,0.001000,Everyone,Sports,0.457143,0,0,...,0,0,0,0,0,0,0,0,0,10.000000
3,Facetune - For Free,0.850,0.001104,0.047331,0.001000,Everyone,Photography,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,10.000000
4,AE 3D Moto 3,0.725,0.000058,0.025176,0.000100,Everyone 10+,Racing,0.185714,0,0,...,0,0,1,0,0,0,0,0,0,1.420433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,WFLA News Channel 8 - Tampa FL,0.700,0.000003,0.013092,0.000010,Everyone,News & Magazines,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,10.000000
8032,Grand Theft Auto: San Andreas,0.850,0.007773,0.025176,0.001000,Mature 17+,Action,0.285714,1,0,...,0,0,0,0,0,0,0,0,0,10.000000
8033,Mix Virtual DJ 2018,0.800,0.000065,0.015106,0.000100,Everyone,Tools,0.442857,0,1,...,0,0,0,0,0,0,0,0,0,10.000000
8034,Cursive Writing Wizard - Handwriting,0.750,0.000083,0.030211,0.001000,Everyone,Education;Education,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,1.418301


In [18]:
validatingSet.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Action,Tools,...,Music,Finance,Board,Racing,Lifestyle,Auto & Vehicles,Music & Video,Parenting,Beauty,Brain Games
0,Shooting King,0.850,5.740715e-03,0.068479,9.999999e-03,Teen,Sports,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Talking Angela,0.675,4.072454e-02,0.051360,1.000000e-01,Teen,Entertainment,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"BestCam Selfie-selfie, beauty camera, photo ed...",0.725,3.871351e-05,0.020141,4.999990e-04,Teen,Beauty,0.432857,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Learn Artificial Intelligence,0.900,5.791434e-07,0.003223,9.999000e-06,Teen,Education,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Co Checker,0.225,2.450222e-07,0.001007,9.990000e-07,Teen,Tools,0.471429,0,1,...,0,0,0,0,0,0,0,0,0,0


In [19]:
KNN.recommendation(trainingSet,validatingSet)

C:\Users\rafal\AppData\Local\Temp\ipykernel_20416\362328838.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["distance"] = [10 for x in range(len(dataset))]
C:\Users\rafal\AppData\Local\Temp\ipykernel_20416\362328838.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["distance"] = distances


,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Action,Tools,...,Finance,Board,Racing,Lifestyle,Auto & Vehicles,Music & Video,Parenting,Beauty,Brain Games,distance
7888,FeaturePoints: Free Gift Cards,0.725,2.702372e-03,0.045317,4.999999e-03,Everyone,Entertainment,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0.518622
2514,FR: My Secret Pets!,0.750,1.746340e-05,0.030211,4.999900e-05,Teen,Entertainment,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,0.520428
6142,FR: My Famous Lover,0.750,4.098554e-06,0.027190,9.999000e-06,Teen,Entertainment,0.285714,0,0,...,0,0,0,0,0,0,0,0,0,0.521150
7641,Quiz for Clash Royale™,0.750,8.435447e-05,0.031219,9.999900e-05,Everyone,Entertainment,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0.522842
1674,Blood Pressure Info,0.725,3.885206e-03,0.006949,9.999999e-03,Everyone,Entertainment,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,0.523700
2563,DJ Electro Mix Pad,0.725,1.187244e-03,0.006647,9.999999e-03,Everyone,Entertainment,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,0.524209
4145,BJ & Jamie,0.725,2.672970e-07,0.016113,9.900000e-08,Everyone,Entertainment,0.190000,0,0,...,0,0,0,0,0,0,0,0,0,0.524258
6491,Elkhart County 4-H Fair,0.725,8.241657e-07,0.041289,4.999000e-06,Everyone,Entertainment,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0.524398
1421,Tattoo Color By Number Draw Book Page Pixel Art,0.750,6.860622e-06,0.024169,9.999900e-05,Everyone,Entertainment,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0.524580
7816,Jarvis artificial intelligent,0.725,5.613236e-06,0.013092,9.999000e-06,Everyone,Entertainment,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.525175
